# Task Management API - Testing & Seeding

This notebook demonstrates how to interact with the Task Management API. It includes steps to:
1.  **Seed Data**: Create initial users, employees, and tasks.
2.  **Test Auth**: Login and get a token.
3.  **Test Endpoints**: CRUD operations for Employees and Tasks.

**Prerequisite**: Ensure the FastAPI server is running locally on `http://localhost:8000`.
Run: `uvicorn app.main:app --reload`

In [ ]:
import requests
import json
import random

BASE_URL = "http://localhost:8000/api/v1"

## 1. Authentication & User Creation

In [ ]:
# Create a test user
username = f"testuser_{random.randint(1000, 9999)}"
password = "password123"

user_data = {
    "username": username,
    "password": password
}

response = requests.post(f"{BASE_URL}/users/", json=user_data)
if response.status_code == 200:
    print(f"User created: {response.json()}")
elif response.status_code == 400:
    print("User already exists, proceeding to login.")
else:
    print(f"Error creating user: {response.text}")

In [ ]:
# Login to get access token
login_data = {
    "username": username,
    "password": password
}

response = requests.post(f"{BASE_URL}/login/access-token", data=login_data)
if response.status_code == 200:
    token = response.json()["access_token"]
    headers = {"Authorization": f"Bearer {token}"}
    print("Login successful! Token received.")
else:
    print(f"Login failed: {response.text}")
    token = None

## 2. Seed Employees

In [ ]:
employees = [
    {"name": "Alice Johnson", "role": "Developer", "email": f"alice_{random.randint(1000,9999)}@example.com"},
    {"name": "Bob Smith", "role": "Manager", "email": f"bob_{random.randint(1000,9999)}@example.com"},
    {"name": "Charlie Brown", "role": "Designer", "email": f"charlie_{random.randint(1000,9999)}@example.com"}
]

created_employees = []

for emp in employees:
    response = requests.post(f"{BASE_URL}/employees/", json=emp)
    if response.status_code == 200:
        data = response.json()
        created_employees.append(data)
        print(f"Created employee: {data['name']} (ID: {data['id']})")
    else:
        print(f"Failed to create employee {emp['name']}: {response.text}")

## 3. Seed Tasks

In [ ]:
if token and created_employees:
    tasks = [
        {"title": "Fix login bug", "description": "Login throws 500 error on wrong password", "status": "Pending", "employee_id": created_employees[0]['id']},
        {"title": "Design homepage", "description": "Create mockups for new homepage", "status": "In Progress", "employee_id": created_employees[2]['id']},
        {"title": "Update documentation", "description": "Add API docs", "status": "Completed", "employee_id": created_employees[0]['id']},
        {"title": "Weekly Report", "description": "Compile weekly stats", "status": "Pending", "employee_id": created_employees[1]['id']}
    ]

    for task in tasks:
        response = requests.post(f"{BASE_URL}/tasks/", json=task, headers=headers)
        if response.status_code == 200:
            print(f"Created task: {task['title']}")
        else:
            print(f"Failed to create task {task['title']}: {response.text}")
else:
    print("Skipping task creation due to missing token or employees.")

## 4. Test Filtering & Listing

In [ ]:
# List all tasks
response = requests.get(f"{BASE_URL}/tasks/")
print(f"Total Tasks: {len(response.json())}")

# Filter by status 'Pending'
response = requests.get(f"{BASE_URL}/tasks/?status=Pending")
print(f"Pending Tasks: {len(response.json())}")

# Filter by Employee
if created_employees:
    emp_id = created_employees[0]['id']
    response = requests.get(f"{BASE_URL}/tasks/?employee_id={emp_id}")
    print(f"Tasks for Employee {emp_id}: {len(response.json())}")